In [8]:
import numpy as np
import pulp
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# Load the data from the CSV file
file_path = '../data/food_nutrition.csv'
nutrition_data = pd.read_csv(file_path)

# Display the data to understand its structure and contents
food_names = nutrition_data['name'].tolist()
nutrition_data.drop(['unit'], axis=1, inplace=True)       # drop the `unit` column
nutrition_data = nutrition_data.reset_index(drop=True)

# Separate the nutrition data for the foods and the lower/upper bounds
food_nutrition = nutrition_data.iloc[2:]
lower_bounds = nutrition_data.iloc[0, 1:]
upper_bounds = nutrition_data.iloc[1, 1:]

# Protein的上下限与WEIGHT有关 => WEIGHT * [.8, 2]
# Fat的上下限与energy有关 => energy * [0.2, 0.35]
WEIGHT = 60             # 60kg body weight
lower_bounds[2] *= WEIGHT
upper_bounds[2] *= WEIGHT
lower_bounds[3] = lower_bounds[1] * 0.2
upper_bounds[3] = upper_bounds[1] * 0.35


YEAR = 30       # 游戏时长（秒）
# 示例数据
# F: 食物中的营养成分矩阵
# L: 营养成分的最小需求
# U: 营养成分的最大限制
# 假设 F 是食物和营养成分的矩阵，L 和 U 分别是营养成分的下界和上界
F = food_nutrition.drop(['name'], axis=1).to_numpy() * 365 * 6  # 每份食物大概600g

SCALAR1, SCALAR2 = 0.75, 1.1    # 用来调整上下界
L = lower_bounds.to_numpy() * SCALAR1 * YEAR * 365  
U = upper_bounds.to_numpy() * SCALAR2 * YEAR * 365


# 创建线性规划问题
lp_prob = pulp.LpProblem("Nutrition_Optimization", pulp.LpMinimize)

# 创建变量
n_foods = F.shape[0]
food_vars = [pulp.LpVariable(f'food_{food_names[i+2]}', lowBound=0, upBound=30, cat='Integer') for i in range(n_foods)]

# 目标函数：最小化食物数量
lp_prob += pulp.lpSum(food_vars)

# 添加营养成分的约束
for j in range(F.shape[1]):
    lp_prob += pulp.lpSum(F[i][j] * food_vars[i] for i in range(n_foods)) >= L[j]
    lp_prob += pulp.lpSum(F[i][j] * food_vars[i] for i in range(n_foods)) <= U[j]

# 求解问题
lp_prob.solve(pulp.PULP_CBC_CMD(msg=0))


# 打印结果
count, non_zero = 0, 0
if pulp.LpStatus[lp_prob.status] == 'Optimal':
    print("找到最优解")
    for var in food_vars:
        count += int(var.varValue)
        non_zero += (1 if var.varValue > 0 else 0)
        print("'" + var.name[5:] + "': " + str(int(var.varValue)) + ",")
        # print(int(var.varValue), "个", var.name)
else:
    print("无法找到最优解")
print('Total food count: ', count, ' Non zero count: ', non_zero)

无法找到最优解
Total food count:  0  Non zero count:  0


In [31]:
import numpy as np
import pulp
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# 定义初始参数：
YEAR, WEIGHT = 30, 60           # 游戏时长（秒）; 60kg body weight
SCALAR1, SCALAR2 = 0.75, 1.1    # 用来调整上下界


def resolve(year=YEAR, weight=WEIGHT, scalar1=SCALAR1, scalar2=SCALAR2):

  # Load the data from the CSV file
  file_path = '../data/food_nutrition.csv'
  nutrition_data = pd.read_csv(file_path)

  # Display the data to understand its structure and contents
  food_names = nutrition_data['name'].tolist()
  nutrition_data.drop(['unit'], axis=1, inplace=True)       # drop the `unit` column
  nutrition_data = nutrition_data.reset_index(drop=True)

  # Separate the nutrition data for the foods and the lower/upper bounds
  food_nutrition = nutrition_data.iloc[2:]
  lower_bounds = nutrition_data.iloc[0, 1:]
  upper_bounds = nutrition_data.iloc[1, 1:]

  # Protein的上下限与WEIGHT有关 => WEIGHT * [.8, 2]
  # Fat的上下限与energy有关 => energy * [0.2, 0.35]
  lower_bounds[2] *= weight
  upper_bounds[2] *= weight
  lower_bounds[3] = lower_bounds[1] * 0.2
  upper_bounds[3] = upper_bounds[1] * 0.35


  # 示例数据
  # F: 食物中的营养成分矩阵
  # L: 营养成分的最小需求
  # U: 营养成分的最大限制
  # 假设 F 是食物和营养成分的矩阵，L 和 U 分别是营养成分的下界和上界
  F = food_nutrition.drop(['name'], axis=1).to_numpy() * 365 * 6  # 每份食物大概600g

  L = lower_bounds.to_numpy() * scalar1 * year * 365  
  U = upper_bounds.to_numpy() * scalar2 * year * 365


  # 创建线性规划问题
  lp_prob = pulp.LpProblem("Nutrition_Optimization", pulp.LpMinimize)

  # 创建变量
  n_foods = F.shape[0]
  food_vars = [pulp.LpVariable(f'{food_names[i+2]}', lowBound=0, upBound=30, cat='Integer') for i in range(n_foods)]

  # 目标函数：最小化食物数量
  lp_prob += pulp.lpSum(food_vars)

  # 添加营养成分的约束
  for j in range(F.shape[1]):
    lp_prob += pulp.lpSum(F[i][j] * food_vars[i] for i in range(n_foods)) >= L[j]
    lp_prob += pulp.lpSum(F[i][j] * food_vars[i] for i in range(n_foods)) <= U[j]

  # 求解问题
  lp_prob.solve(pulp.PULP_CBC_CMD(msg=0))


  # 打印结果
  count, non_zero = 0, 0
  if pulp.LpStatus[lp_prob.status] == 'Optimal':
    print("找到最优解")
    return food_vars
    # for var in food_vars:
    #     count += int(var.varValue)
    #     non_zero += (1 if var.varValue > 0 else 0)
    #     print("'" + var.name + "': " + str(int(var.varValue)) + ",")

    # print('Total food count: ', count, ' Non zero count: ', non_zero)
    # return True
  else:
    print("无法找到最优解")
    return



# 多次求解线性规划问题
y = 15                  # 起始年份
interval = 2            # 每interval年检查一次健康状况
yearly_food_vars = []   # 每一个年份的 线性规划的解。共 int(YEAR / interval) 年
while y <= YEAR:
  food_vars = resolve(year=y)
  if food_vars:
    yearly_food_vars.append(food_vars)
    y += interval
  else:
    print('在第' + str(y) + '年无解')
    break



# 通常来说，游戏刚开始的几秒是很难找到可行解的，所以需要设置一个 y值，使得第 y 秒之后开始有可行解
y = 15                           # 重新设置起始年份
years = []                       # 输出结果
food_names = []                  # 输出结果
food_counts = []                 # 输出结果
if len(yearly_food_vars) == int((YEAR - y) / interval + 1):
  for food_vars in yearly_food_vars:
    years.append(y)
    food_names.append([var.name for var in food_vars])
    food_counts.append([int(var.varValue) for var in food_vars])
    y += interval

food_counts = np.array(food_counts)
food_counts = np.transpose(food_counts).tolist()

yearly_solutions = pd.DataFrame.from_records(data=food_counts, columns=years, index=food_names[0])



找到最优解
找到最优解
找到最优解
找到最优解
找到最优解
找到最优解
找到最优解
找到最优解
29 8


,15,17,19,21,23,25,27,29
watermelon,0,0,0,0,0,0,0,0
apple,0,0,0,0,0,0,0,0
banana,0,3,0,3,0,2,3,5
avocado,0,0,0,0,0,0,0,0
broccoli,1,1,1,1,1,1,1,1
pink_salmon,0,0,0,0,0,2,1,0
chicken,0,0,0,0,1,0,0,0
beef,2,0,1,1,2,0,1,0
arugula,1,1,1,1,2,2,3,0
bread,0,0,0,0,0,0,0,0
